In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from src.network_diffusion_model import NDM
from src.FKPP_model import FKPP
from src.find_optimal_timepoint import find_optimal_timepoint


## Loading files

First we will load in some key files: the reference list and the connectome. The reference list should contain labels for each node in the connectome, in the same order as they appear in the connectome matrix. These files are all we need to simulate pathology spread using network spreading models.

The region names in the reference list should be in the format: "region_L" or "region_R", where "L" and "R" denote left and right hemispheres, respectively.

Let's look at the example files in "tutorial_data":

In [ ]:
# load in the connectome
connectome_fname = "tutorial_data/tractography.csv"
connectome = pd.read_csv(connectome_fname, header=None).values

# load in the reference list
ref_list_df = pd.read_csv("tutorial_data/RegionList.csv")
ref_list = ref_list_df.region_name.to_list() # needs to be converted to a list
print(ref_list)

 We can check that the number of regions in the connectome matches the number of regions in the reference list


In [ ]:
print(connectome.shape[0] == len(ref_list))

## Initialising the model

We will start by running the network diffusion model with the example data. First we need to set the time points for which we want to simulate pathology spread. Here we will simulate from time 0 to 25 in 1000 steps.

In [ ]:
t = np.linspace(0,25,1000)

Now we will set up our model, using the connectome file and reference list we loaded earlier.

We set gamma, the spreading parameter, to 1 here. Since we are modelling on an arbitrary timescale the value of gamma doesn't matter. We also set the seed region to "Entorhinal", meaning that pathology will be initialised at a value of 1 in both the left and right entorhinal cortices, and 0 elsewhere.


In [ ]:
ndm = NDM(connectome_fname,
          gamma=1,
          t=t,
          ref_list=ref_list,
          seed_region = "Entorhinal")  

We simulate pathology spread using the `run_NDM` method. We can check the size of the output, it should be the number of brain regions by the number of time points we specified earlier.

In [ ]:
output = ndm.run_NDM()

print(output.shape) # n_regions x n_timepoints

## Visualising the output

We can visualise the simulated pathology spread over time using the `plot_time_series` function. This will plot the predicted pathology accumulation over time for each brain region, with different colours for each region. It is important to check that the time points extend far enough for the pathology to reach a steady state across the network.

In [ ]:
from src.plot_functions import plot_times_series

plot_times_series(output, t)

We can do the same for the FKPP model. This is similar to the NDM, but contains an additional term that simulates local production of pathology in each region.

The balance between spreading and local production is controlled by the alpha parameter. Here we set alpha to 0.5, meaning that spreading and local production contribute equally to pathology accumulation. We also set gamma to 1 as before, and use the same time points and seed region.

In [ ]:
fkpp = FKPP(connectome_fname,
            alpha=0.5,
            gamma=1,
            t=t,
            ref_list=ref_list,
            seed_region = "Entorhinal")

In [ ]:
output_fkpp = fkpp.run_FKPP()

plot_times_series(output_fkpp, t)

You can try adjusting the alpha parameter and the seed region to see how this affects the spread dynamics.

## Fitting the model to some target data

Typically, we want to fit the model to some empirical data, for example regional pathology measurements from patients. We can then examine the optimal parameters that provide the best fit to our data. 
Here we will load in some example target data and fit the model to this data. The target data comes from tau PET measurements, averaged across patients on the Alzheimer's disease spectrum.

It is important to check that the ordering of the regions in the target data matches those in the reference list.

In [ ]:
df = pd.read_csv("tutorial_data/pathology.csv", names=['region','pathology'], header=0)
df

We want to extract the pathology values and normalise them to a range of 0 to 1 for fitting. 

In [ ]:
target_pathology = df['pathology'].values

def normalise_data(data):
    return (data - data.min()) / (data.max() - data.min())

target_pathology = normalise_data(target_pathology)
target_pathology

We now have our target pathology data ready for fitting!

### Fitting the network diffusion 
For the NDM model, we simply find the seed region that provides the best fit to the target data. The `optimise_seed_region` method returns the sum-of-squared-errors (SSE) between the model output and target data for each seed region, as well as the optimal seed that gives the lowest SSE.

In [ ]:
res, optimal_params = ndm.optimise_seed_region(target_data=target_pathology)

print("optimal seed region", optimal_params, "\n")

print(res.sort_values(by='SSE').head())

We can see that the seeds that provide the best fit to the data are mostly in the temporal lobe, as we would expect.

Now we can run the model again using the optimal seed region to visualise the fit to the data.

The `find_optimal_timepoint` function finds the time point at which the model output best fits the target data, returning the index of this time point, the predicted pathology values at this time point, and the corresponding SSE.

In [ ]:
ndm = NDM(connectome_fname,
          gamma=1,
          t=t,
          ref_list=ref_list,
          seed_region = optimal_params['seed'])

ndm_output = ndm.run_NDM()
min_idx, ndm_prediction, SSE = find_optimal_timepoint(ndm_output, target_pathology)

Let's plot the predicted pathology values against the target data at this optimal time point. You could also use a package like ggseg to visualise the spatial distribution of pathology across the brain regions.

In [ ]:
plt.scatter(target_pathology, ndm_prediction)
plt.xlabel("Target Pathology")
plt.ylabel("Model Prediction")
plt.title("NDM Model Fit to Target Pathology")

print("correlation between model prediction and pathology = ", np.corrcoef(target_pathology, ndm_prediction)[0,1])

### Fitting the FKPP model

The `optimise_fkpp_params` method optimises both the seed region and the alpha parameter for the FKPP model. It returns the optimal parameters that provide the best fit to the target data. This takes a little longer to run since it needs to optimise alpha for each seed region, but should only take a few minutes.

In [ ]:
fkpp = FKPP(connectome_fname,
            gamma=1,
            t=t,
            ref_list=ref_list)
res, optimal_params = fkpp.optimise_fkpp_params(target_data=target_pathology)

print("optimal parameters", optimal_params, "\n")

Again, we can run the optimised model to get the predicted pathology values at the optimal time point.

In [ ]:
fkpp = FKPP(connectome_fname,
            alpha=optimal_params['alpha'],
            gamma=1,
            t=t,
            ref_list=ref_list,
            seed_region = optimal_params['seed'])

fkpp_output = fkpp.run_FKPP()
min_idx, fkpp_prediction, SSE = find_optimal_timepoint(fkpp_output, target_pathology)

In [ ]:
plt.scatter(target_pathology, fkpp_prediction)
plt.xlabel("Target Pathology")
plt.ylabel("Model Prediction")
plt.title("FKPP Model Fit to Target Pathology")

print("correlation between model prediction and pathology = ", np.corrcoef(target_pathology, fkpp_prediction)[0,1])

## Weighted FKPP model
The third model in the toolbox is the weighted-FKPP, which extends the FKPP by allowing for different rates of pathology production in each brain region. This allows us to model regional variations in pathology vulnerability across the brain. We can use this model to test hypotheses about different factors that may influence pathology production, such as gene expression or connectivity properties.

In practise, it is very similar to running the FKPP model, but we need to provide a `weights` array that contains the relative production rates for each brain region. In this example we will use regional amyloid-PET values as weights, since amyloid pathology is thought to influence tau pathology production in Alzheimer's disease. We can test whether including these weights improves the model fit to the target tau-PET data.

Again, it is important to check that the ordering of the regions in the weights data matches those in the reference list.



In [ ]:
amyloid = pd.read_csv("tutorial_data/regional_amyloid.csv", names=['region','amyloid'], header=0)

weighted_fkpp = FKPP(connectome_fname,
            gamma=1,
            t=t,
            ref_list=ref_list,
            weights=amyloid['amyloid'].values)

res, optimal_params = weighted_fkpp.optimise_fkpp_params(target_data=target_pathology)

print("optimal parameters", optimal_params, "\n")

optimal_weighted_fkpp = FKPP(connectome_fname,
            alpha=optimal_params['alpha'],
            gamma=1,
            t=t,
            ref_list=ref_list,
            weights=amyloid['amyloid'].values,
            seed_region = optimal_params['seed'])

optimal_weighted_fkpp_output = optimal_weighted_fkpp.run_FKPP()
min_idx, weighted_fkpp_prediction, SSE = find_optimal_timepoint(optimal_weighted_fkpp_output, target_pathology)


In [ ]:
plt.scatter(target_pathology, weighted_fkpp_prediction)
plt.xlabel("Target Pathology")
plt.ylabel("Model Prediction")
plt.title("weighted FKPP Model Fit to Target Pathology")

print("correlation between model prediction and pathology = ", np.corrcoef(target_pathology, weighted_fkpp_prediction)[0,1])

## Model comparison

We can use the ModelSelection class to compare the two models based on their residuals. Here we use the corrected Akaike Information Criterion (AICc) to compare the NDM and FKPP models. The model with the lower AICc value is considered to have a better fit to the data, taking into account the number of parameters in each model. The AICc weights can be interpreted as the probability that the corresponding model is the best model for the data among the set of candidate models.

In [ ]:
from src.ModelSelection import ModelSelection

# calculate the residuals for each model, needs to be sized (n_obs, n_models)
residuals = np.array([target_pathology - ndm_prediction,
                        target_pathology - fkpp_prediction,
                        target_pathology - weighted_fkpp_prediction]).T

# the FKPP has 3 parameters (alpha, seed, time), NDM has 2 (seed, time)
results = ModelSelection(n_dof=[2,3,3], criterion="AICc", model_names=["NDM", "FKPP", "weighted-FKPP"])(residuals)
print(results)

We can see from these results that the weighted-FKPP model has the lowest AICc value and the highest AICc weight, indicating that it provides a better fit to the target pathology data compared to the other models. This suggests that weighting pathology production in the model by amyloid burden improves its ability to capture the observed patterns of pathology accumulation.

# Other features

### Filtering output to specific regions
`cortical_idx` allows you to choose a particular subset of regions to use for model fitting. For example, you may want to fit your model to cortical regions in your target data (e.g. to avoid contamination from off-target binding in a PET tracer), while still simulating pathology spread through the full connectome, including subcortical connections. 

Let's try this with our example data:

In [ ]:
# just selecting the indices of the cortical regions
cortical_idx = list(range(0,34)) + list(range(49, 83))

# Use cortical_idx to index ref_list
ref_list_subset = [ref_list[i] for i in cortical_idx]
print(ref_list_subset)

filter target pathology to only cortical regions and renormalise:


In [ ]:
target_pathology_cortical = target_pathology[cortical_idx]
target_pathology_cortical = normalise_data(target_pathology_cortical)

Now we can run the NDM using the `cortical_idx` flag so that we just output the cortical regions to match the target data.

However, the model is still simulating pathology dynamics with the full connectome, including connections to the subcortex.

In [ ]:
ndm = NDM(connectome_fname,
          gamma=1,
          t=t,
          ref_list=ref_list,
          seed_region = "Inferiortemporal",
          cortical_idx = cortical_idx)

ndm_output = ndm.run_NDM()
min_idx, ndm_prediction, SSE = find_optimal_timepoint(ndm_output, target_pathology_cortical)

We can see that the model fit is improved now we have removed the subcortical regions that are affected by off-target binding of the tau-PET tracer:

In [ ]:
plt.scatter(target_pathology_cortical, ndm_prediction)
plt.xlabel("Target Pathology")
plt.ylabel("Model Prediction")
plt.title("NDM Model Fit to Target Pathology")

print("correlation between model prediction and pathology = ", np.corrcoef(target_pathology_cortical, ndm_prediction)[0,1])

### Lateral seeding

In the standard seeding paradigm, we initialise pathology in a particular region in both hemispheres (e.g. "Inferiortemporal_L", and "Inferiortemporal_R"). When we optimise the seed region, we find the best bilateral seed.

However, sometimes we may be looking at asymmetric data where we want to seed pathology in one hemisphere only. We can do this with the `lateral_seeding` flag:

In [ ]:
ndm = NDM(connectome_fname,
          gamma=1,
          t=t,
          ref_list=ref_list,
          seed_region = "Inferiortemporal_L", # now we can specify our seed region in a particular hemisphere
          lateral_seeding=True)

ndm_output_lateral_seed = ndm.run_NDM()
plot_times_series(ndm_output_lateral_seed, t)

If we optimise the seed region in lateral seeding mode, we compare the performance of each brain region in both hemispheres separately:

In [ ]:
res, optimal_params = ndm.optimise_seed_region(target_data=target_pathology)
optimal_params
print(res.sort_values(by='SSE').head())

For this data, the model fit is better using *bilateral seeding*, as the data is fairly symmetric accross hemispheres.

### Custom initialisation

Finally, we can remove the need for seeds entirely and simply initialise the model with a vector of our choice: `x0`. This means that we set the initial pathology vector when t = 0.

We can use this to model longitudinal data, for example setting `x0` to an initial observation and fitting to target data at a subsequent timepoint.

Here, we will just use a random vector to illustrate the concept:



In [ ]:
random_initial_vector = np.random.rand(connectome.shape[0])
ndm = NDM(connectome_fname,
          gamma=1,
          t=t,
          ref_list=ref_list,
          x0 = random_initial_vector)  # set initial pathology vector

ndm_output_x0 = ndm.run_NDM()
plot_times_series(ndm_output_x0, t)

From the figure we can see how the predicted pathology at t = 0 is now randomly distributed from 0-1.